# Trace AI
---

## Preamble

In [33]:
# Install libraries
# pip install --upgrade google-cloud-aiplatform
# pip install PyPDF2

In [73]:
# Authentication to Google Cloud
# gcloud auth application-default login

In [72]:
# Import Libraries

import pandas as pd
import numpy as np

import json
import PyPDF2
import vertexai
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory, ChatSession

In [ ]:
# Initiate Connection with GCP Vertex AI Gemini Model

vertexai.init(project="tutorial-house", location="us-central1")

safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH}

model_gemini = GenerativeModel(model_name="gemini-1.5-flash-002",
                               safety_settings=safety_settings)

## 1. Read Document

Disini kami bla bla

In [65]:
# Path to your PDF file
file_path = "Contoh Proposal Penawaran Pengadaan Alat Tulis Kantor.pdf"

# Read and extract text from PDF
document_text = ""
with open(file_path, "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        document_text += page.extract_text() + "\n"

print(document_text)

Proposal Penawaran Pengadaan Alat Tulis Kantor (ATK)  
Diajukan oleh PT. Alat Kantor Nusantara  
Untuk Kementerian Keuangan
 
1. Latar Belakang  
Sebagai penyedia perlengkapan kantor terkemuka, PT. Alat Kantor Nusantara siap 
memenuhi kebutuhan ATK untuk mendukung operasional di Kementerian Keuangan. 
Kami menyediakan produk berkualitas tinggi dengan harga yang bersaing, pengiriman 
tepat waktu, serta dukungan layanan purna jual.  
 
2. Penawaran Produk  
Kami menawarkan pengadaan berbagai jenis ATK sebagai berikut:  
No. Nama Barang  Satuan  Jumlah  Harga Satuan 
(Rp)  Total Harga (Rp)  
1 Pulpen Gel  Pcs 2000  6,000,00  12,000,000,00  
2 Pensil Mekanik  Pcs 1000  6,000,00  6,000,000,00  
3 Kertas HVS A4 (80gsm)  Rim 400  41,000,00  16,400,000,00  
4 Kertas HVS A3 (70gsm)  Rim 150  70,000,00  10,500,000,00  
5 Penggaris Logam 30 
cm Pcs 200  4,250,00  850,000,00  
6 Amplop Coklat A4  Pack  150  30,000,00  4,500,000,00  
7 Map Snelhekter Plastik  Pack  300  11,000,00  3,300,000,00  
8 

## 2. Generate document summary

In [69]:
# Design AI Agent to Generate List of Items based on the Inputted Documents

prompt_summary = """Keluarkan summary dari document tersebut"""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

response_summary = model_gemini.generate_content(
    [document_text, prompt_summary],
    generation_config=generation_config,
)

response_summary.text

'PT. Alat Kantor Nusantara mengajukan penawaran pengadaan Alat Tulis Kantor (ATK) kepada Kementerian Keuangan senilai Rp 160.727.500.  Penawaran mencakup 30 jenis ATK dengan rincian jumlah dan harga satuan masing-masing item.  Layanan tambahan meliputi garansi produk 6 bulan, pengiriman cepat (maksimal 3 hari kerja), dan layanan purna jual.  PT. Alat Kantor Nusantara berkomitmen memberikan layanan pengadaan ATK yang optimal dan berkualitas tinggi dengan harga kompetitif.\n'

## 3. Generate List of Items

In [71]:
# Design AI Agent to Generate List of Items based on the Inputted Documents

prompt_items = """Keluarkan list barang dan jasa pengadaan dari dokumen tersebut dalam bentuk table dengan format sebagai berikut; nama barang, jumlah, harga satuan, harga total. Keluarkan dalam bentuk json dengan nama 'data'"""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
    "response_mime_type": "application/json"
}

response_item = model_gemini.generate_content(
    [document_text, prompt_items],
    generation_config=generation_config,
)

# Parse JSON string
parsed_data = json.loads(response_item.text)

# Convert data to DataFrame
df = pd.DataFrame(parsed_data["data"])

# Display the DataFrame
df

,nama_barang,jumlah,harga_satuan,harga_total
0,Pulpen Gel,2000,6000,12000000
1,Pensil Mekanik,1000,6000,6000000
2,Kertas HVS A4 (80gsm),400,41000,16400000
3,Kertas HVS A3 (70gsm),150,70000,10500000
4,Penggaris Logam 30 cm,200,4250,850000
5,Amplop Coklat A4,150,30000,4500000
6,Map Snelhekter Plastik,300,11000,3300000
7,Staples dan Isi,200,15000,3000000
8,Gunting Besar,150,50000,7500000
9,Korektor Pita,400,5100,2040000


## 4. Retrieve Similar Items from Database

In [48]:
## RA

## 5. Compare Similar Item Pricing and Generate Reasonings

## 6. Further Interact with the document and the data

In [50]:
def model_generate_content(model: ChatSession, query: str, config):
    
    try:
        return model.send_message(query, generation_config=config).text
    except Exception as err:
        print(err)
        return "Sorry.. Service temporary unavailable."

In [57]:
model_chat = model_gemini.start_chat(response_validation=False)

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
    "response_mime_type": "application/json"
}

start_template = f"""Anda adalah chatbot yang berfungsi untuk menjawab pertanyaan user terkait dokumen dan pengadaan barang dan jasa berkaitan dengan sumber yang diberikan

Sumber Dokumen: {document_text}
List barang dan jasa: {response_item.text}

"""

chat_1 = 'Apa item yang paling mahal?'

chat = start_template + chat_1

result_ai = model_generate_content(model_chat, chat, config=generation_config)

result_ai


'Berdasarkan data yang diberikan, item yang paling mahal adalah Binder dengan harga total Rp 30.000.000.\n'

In [58]:
chat_2 = 'Apa item yang paling mahal selain item tersebut?'

chat = chat_2

result_ai = model_generate_content(model_chat, chat, config=generation_config)

result_ai

'Item yang paling mahal kedua setelah Binder adalah Kertas HVS A4 (80gsm) dengan harga total Rp 16.400.000.\n'